<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/osaka_unknown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アップロード

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print(
        'User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])
        )
    )

# 設定

In [ ]:
# 大阪南部

mid = "11UCAA68tszbyxJL9y_tng0QYcRdtCJ63"
lid = ""

url = f"https://www.google.com/maps/d/kml?mid={mid}" if lid == "" else f"https://www.google.com/maps/d/kml?mid={mid}&lid={lid}"

In [ ]:
url

In [ ]:
# 抽出用enb-LCID

enb = [443135]
lcid = [7]

# プログラム

## 準備

In [ ]:
import pathlib
import zipfile

from lxml import etree

In [ ]:
import pandas as pd
import requests

In [ ]:
import folium
from folium.features import DivIcon

In [ ]:
def fetch_file(url, fn):

    p = pathlib.Path(fn)

    r = requests.get(url)
    r.raise_for_status()

    with p.open(mode="wb") as fw:
        fw.write(r.content)
    return p

In [ ]:
colors = [
    "black",
    "red",
    "green",
    "blue",
    "darkred",
    "darkgreen",
    "darkblue",
    "lightred",
    "lightgreen",
    "lightblue",
    "pink",
    "orange",
    "cadetblue",
    "beige",
    "gray",
    "darkpurple",
    "lightgray",
    "white",
]

In [ ]:
# マイマップからデータ抽出

def get_mymap(url):

    ns = {"kml": "http://www.opengis.net/kml/2.2"}

    p = fetch_file(url, "rakuten.kmz")

    with zipfile.ZipFile(p) as existing_zip:
        existing_zip.extractall("kmz")

    tree = etree.parse("./kmz/doc.kml")

    data = []

    for i in tree.xpath("//kml:Placemark", namespaces=ns):

        d = {}

        d["場所"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
        d["geometry"] = "".join(
            i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
        ).strip()

        for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

            k = j.attrib["name"].strip()
            v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

            d[k] = v

        data.append(d)

    df0 = pd.DataFrame(data)

    df0[["経度", "緯度", "標高"]] = df0["geometry"].str.split(",", expand=True)

    # 文字から数値に変換
    df0["経度"] = pd.to_numeric(df0["経度"])
    df0["経度"] = pd.to_numeric(df0["経度"])

    # 緯度・経度が欠損の場合は削除
    df0.dropna(subset=["緯度", "経度"], how="any", inplace=True)

    df0["場所"] = df0["場所"].str.replace("【.+】", "", regex=True)

    df0["eNB-LCID"] = df0["eNB-LCID"].str.strip()

    df0["eNB-LCID"] = df0["eNB-LCID"].mask(df0["eNB-LCID"] == "")

    df1 = df0[["場所", "eNB-LCID", "緯度", "経度"]].copy()

    return df1

In [ ]:
# 電測ポイントを地図表示

def map_make(df):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    # map = folium.Map(location=[lat, lon], zoom_start=14)

    map = folium.Map(tiles=None, location=[lat, lon], zoom_start=14)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
    ).add_to(map)

    for _, r in df.iterrows():

        cid = r["rnc"] if r["rnc"] < 19 else 0

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=colors[cid],
            fill=True,
            tooltip=r["id"],
        ).add_to(map)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["id"]} TA:{r["ta"]}',
        ).add_to(map)

    return map

## TowerCollector

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"], dtype={"ta": str}).dropna(
    how="all", axis=1
)

df0

In [ ]:
# 日時変換
df0["measured_at"] = df0["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

# TAが欠損
df0["ta"] = df0["ta"].fillna("-")

df0.dropna(how="any", subset=["short_cell_id", "rnc"], inplace=True)

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

In [ ]:
df1["short_cell_id"] = df1["short_cell_id"].astype(int)
df1["rnc"] = df1["rnc"].astype(int)

In [ ]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [ ]:
# dbmが最大
df2 = df1.groupby(["short_cell_id", "rnc"])["dbm"].max().reset_index()
df2

In [ ]:
df3 = pd.merge(df1, df2)

df3

# 地図

## 全部

In [ ]:
map0 = map_make(df1)
map0.save("map0.html")
map0

## 基地局付近

In [ ]:
map1 = map_make(df3)
map1.save("map1.html")
map1

In [ ]:
df4 = get_mymap(url).dropna(subset=["eNB-LCID"])

In [ ]:
df4

## 未確認

In [ ]:
df4[["eNB", "LCID"]] = df4["eNB-LCID"].str.split("-", expand=True)
df4["LCID"] = df4["LCID"].str.split(",")

In [ ]:
df5 = df4.explode("LCID")
df5["id"] = df5["eNB"] + "-" + df5["LCID"]

In [ ]:
df6 = df3[~df3["id"].isin(df5["id"])]
df6

In [ ]:
if len(df6) > 0:
    map2 = map_make(df6)
    map2.save("map2.html")

else:
    print("データなし")

In [ ]:
map2

# 抽出

In [ ]:
df1[df1["short_cell_id"].isin(enb)]

df7 = df1[df1["short_cell_id"].isin(enb) & df1["rnc"].isin(lcid)]
df7

In [ ]:
if len(df7) > 0:
    map3 = map_make(df7)
    map3.save("map3.html")

else:
    print("データなし")

In [ ]:
map3